<a href="https://colab.research.google.com/github/falgunip02/IPD/blob/main/LocationSuggest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

class CropLocationSuggester:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.df = None
        self.label_encoder = LabelEncoder()

    def load_data(self):
        try:
            self.df = pd.read_csv(self.csv_file, names=['croptype', 'labourprice', 'location'])
        except FileNotFoundError:
            print("Error: CSV file not found.")
            return False
        except pd.errors.EmptyDataError:
            print("Error: CSV file is empty.")
            return False
        return True

    def prepare_data(self):
        if self.df is None:
            return False
        self.df['croptype'] = self.label_encoder.fit_transform(self.df['croptype'])
        self.df['location'] = self.label_encoder.fit_transform(self.df['location'])
        return True

    def filter_data(self, crop_type):
        filtered_data = self.df[self.df['croptype'] == crop_type]
        return filtered_data

    def train_model(self, X, y):
        regressor = RandomForestRegressor()
        regressor.fit(X, y)
        return regressor

    def predict_location(self, regressor, locations):
        predicted_prices = {loc: regressor.predict([[loc]])[0] for loc in locations}
        best_location = min(predicted_prices, key=predicted_prices.get)
        return self.label_encoder.inverse_transform([best_location])[0]

    def suggest_location(self, crop_type):
        if not self.load_data():
            return "Error: Unable to load data."
        if not self.prepare_data():
            return "Error: Unable to prepare data."
        filtered_data = self.filter_data(crop_type)
        if filtered_data.empty:
            return "No data available for the given crop type."
        X = filtered_data[['location']]
        y = filtered_data['labourprice']
        regressor = self.train_model(X, y)
        locations = filtered_data['location'].unique()
        best_location = self.predict_location(regressor, locations)
        return best_location

# Example usage:
csv_file = "crop_data.csv"
suggester = CropLocationSuggester(csv_file)

# Example: Suggest location for Rice
crop_type = "Rice"
best_location = suggester.suggest_location(crop_type)
print(f"The best location for {crop_type} is: {best_location}")

The best location for Rice is: No data available for the given crop type.


In [20]:
import csv
import random

# Function to generate random locations in India
def generate_random_location():
    locations = ["Rajasthan", "Uttar Pradesh", "Punjab", "Madhya Pradesh", "Haryana"]
    return random.choice(locations)

# Function to generate random crop types
def generate_random_crop_type():
    crop_types = ["Wheat", "Rice", "Cotton", "Soybean", "Mustard", "Sugarcane", "Maize", "Barley", "Lentils", "Millet"]
    return random.choice(crop_types)

# Function to generate random prices
def generate_random_price():
    return random.randint(400, 800)

# Function to generate random wages
def generate_random_wages():
    return random.randint(800, 1200)

# Function to generate random distances
def generate_random_distance():
    return random.randint(100, 500)

# Function to generate random travel times
def generate_random_travel_time():
    return random.randint(3, 8)

# Generate Farm Laborers Data
with open('farm_laborers_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['ID', 'Location', 'Preferred_Work_Location']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(1, 101):
        writer.writerow({'ID': i, 'Location': generate_random_location(), 'Preferred_Work_Location': generate_random_location()})

# Generate Crop Season Data
with open('crop_season_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['Crop_Type', 'Location', 'Expected_Price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(1, 200):
        writer.writerow({'Crop_Type': generate_random_crop_type(), 'Location': generate_random_location(), 'Expected_Price': generate_random_price()})

# Generate Job Opportunities Data
with open('job_opportunities_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['Job_ID', 'Crop_Type', 'Location', 'Wages']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(1, 101):
        writer.writerow({'Job_ID': i, 'Crop_Type': generate_random_crop_type(), 'Location': generate_random_location(), 'Wages': generate_random_wages()})

# Generate Price Data
with open('price_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['Crop_Type', 'Location', 'Price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(1, 101):
        writer.writerow({'Crop_Type': generate_random_crop_type(), 'Location': generate_random_location(), 'Price': generate_random_price()})

# Generate Migration Routes Data
with open('migration_routes_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['From_Location', 'To_Location', 'Distance', 'Travel_Time']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(1, 101):
        writer.writerow({'From_Location': generate_random_location(), 'To_Location': generate_random_location(), 'Distance': generate_random_distance(), 'Travel_Time': generate_random_travel_time()})


In [27]:
import pandas as pd

# Load the data from CSV files
farm_laborers_data = pd.read_csv('farm_laborers_data.csv')
job_opportunities_data = pd.read_csv('job_opportunities_data.csv')
price_data = pd.read_csv('price_data.csv')

# Define array of crop types
crop_types = ['Wheat', 'Rice', 'Cotton', 'Soybean']

# Iterate through each crop type
for selected_crop_type in crop_types:
    print(f"Processing crop type: {selected_crop_type}")
    print("----------------------------------------")

    # Filter job opportunities data for selected crop type
    filtered_job_opportunities = job_opportunities_data[job_opportunities_data['Crop_Type'] == selected_crop_type]

    # Calculate weighted scores
    grouped_job_opportunities = filtered_job_opportunities.groupby('Location')
    average_wages = grouped_job_opportunities['Wages'].mean()
    average_prices = price_data[price_data['Crop_Type'] == selected_crop_type].groupby('Location')['Price'].mean()

    # Drop locations with NaN values in either average wages or average prices
    weighted_scores = (0.7 * average_wages) + (0.3 * average_prices)
    weighted_scores = weighted_scores.dropna()

    # Recommend the location with the highest weighted score
    if not weighted_scores.empty:
        best_location = weighted_scores.idxmax()
        print(f"The best location for {selected_crop_type} is: {best_location}")
    else:
        print(f"No data available for {selected_crop_type}.")
    print()


Processing crop type: Wheat
----------------------------------------
The best location for Wheat is: Punjab

Processing crop type: Rice
----------------------------------------
The best location for Rice is: Madhya Pradesh

Processing crop type: Cotton
----------------------------------------
The best location for Cotton is: Uttar Pradesh

Processing crop type: Soybean
----------------------------------------
The best location for Soybean is: Uttar Pradesh

